In [1]:
# import needed
from bayes_opt import BayesianOptimization
from cloud_pipeline import CloudPipeline
from cloud_pipeline.config import ARG_KEYS
import numpy as np


In [2]:
# instantiate cloud_pipeline
cp = CloudPipeline()

In [3]:
# cleanup
if cp.vnf_existed():
    cp.vnf_cleanup()

# genarate load
if not cp.load_existed():
    load_gen = cp.load_gen()


/usr/lib/python2.7/dist-packages/urllib3/util/ssl_.py:315: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/lib/python2.7/dist-packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [4]:
# Bounded region of parameter space
parameter_bounds = dict(zip(ARG_KEYS, [(-1, 1)] * len(ARG_KEYS)))

parameter_bounds

{'w_cpu_p': (-1, 1),
 'w_disk': (-1, 1),
 'w_frequency': (-1, 1),
 'w_idle_p': (-1, 1),
 'w_iowait_p': (-1, 1),
 'w_kernel_p': (-1, 1),
 'w_ram': (-1, 1),
 'w_user_p': (-1, 1)}

In [5]:
optimizer = BayesianOptimization(
    f=cp.benchmark_run,
    pbounds=parameter_bounds,
    random_state=1,
)

In [ ]:
optimizer.maximize(
    init_points=2,
    n_iter=20,
)

|   iter    |  target   |  w_cpu_p  |  w_disk   | w_freq... | w_idle_p  | w_iowa... | w_kern... |   w_ram   | w_user_p  |
-------------------------------------------------------------------------------------------------------------------------


In [14]:
#for i, res in enumerate(optimizer.res):
#    print("Iteration {}: \n\t{}".format(i, res))

len(optimizer.res)


15

# Serilization

In [21]:
optimizer.res[0]

{'params': {'w_cpu_p': -0.165955990594852,
  'w_disk': 0.4406489868843162,
  'w_frequency': -0.9997712503653102,
  'w_idle_p': -0.39533485473632046,
  'w_iowait_p': -0.7064882183657739,
  'w_kernel_p': -0.8153228104624044,
  'w_ram': -0.6274795772446582,
  'w_user_p': -0.3088785459139045},
 'target': 1616.7473464993066}

In [6]:
import json

#res = optimizer.res
log_path = "/var/lib/cloud_pipeline/log.json"

def serilization(res_dict, log_path=log_path):
    json_logs = json.dumps()
    with open(log_path, "w") as j:
        j.write(json_logs)

# Unserilization

In [56]:
optimizer1 = BayesianOptimization(
    f=cp.benchmark_run,
    pbounds=parameter_bounds,
    random_state=1,
)


In [7]:
def unserilization(optimzer, log_path=log_path):
    with open(log_path, "r") as j:
        logs = json.loads(j.read())
    log_dict = {}
    log_dict["params"] = []
    log_dict["target"] = []
    for iteration in logs:
        optimzer.register(
            params=iteration["params"],
            target=iteration["target"],
        )


In [60]:
optimizer1.res[-1]

{'params': {'w_cpu_p': 0.9750692655000017,
  'w_disk': -0.9414383839381675,
  'w_frequency': -0.8625900540578666,
  'w_idle_p': 0.9559815816629342,
  'w_iowait_p': 0.5088451292635974,
  'w_kernel_p': 0.40212223768049493,
  'w_ram': 0.9876815146033604,
  'w_user_p': 0.11764165429276385},
 'target': 155.8629217310591}

## Contined Training

In [8]:
optimizer2 = BayesianOptimization(
    f=cp.benchmark_run,
    pbounds=parameter_bounds,
    random_state=1,
)
unserilization(optimizer2, log_path)

In [ ]:
optimizer2.maximize(
    #init_points=2,
    n_iter=20,
)
log_path2 = "/var/lib/cloud_pipeline/log_2.json"
serilization(optimizer2, log_path2)

|   iter    |  target   |  w_cpu_p  |  w_disk   | w_freq... | w_idle_p  | w_iowa... | w_kern... |   w_ram   | w_user_p  |
-------------------------------------------------------------------------------------------------------------------------
|  1        |  155.9    |  0.9751   | -0.9414   | -0.8626   |  0.956    |  0.5088   |  0.4021   |  0.9877   |  0.1176   |
|  2        |  155.9    |  0.9751   | -0.9414   | -0.8626   |  0.956    |  0.5088   |  0.4021   |  0.9877   |  0.1176   |
